In [1]:
## This code chunk was hidden in the original document, but was executed in the background
knitr::opts_chunk$set(results="hide", message=FALSE, warning=FALSE, fig.show="hide", echo=TRUE)

<h1>
Annotating Genomic Variants
</h1>
-   [Background](#background)
-   [Set Up](#setup)
-   [Exploring variants in the TRPV gene family](#TRPV)
    -   [Examine header data in a vcf file](#TRPV_header)
    -   [Convert gene symbols to gene ids](#TRPV_symbols)
    -   [Create gene ranges](#TRPV_ranges)
    -   [Extract variant subsets](#TRPV_subset)
    -   [Variant location in the gene model](#TRPV_locate)
    -   [Amino acid coding changes in non-synonymous variants](#TRPV_predict)
-   [Annotating with the ensemblVEP package](#vep)
-   [Exploring Package Content](#content)
-   [sessionInfo()](#session)

<h2 id="background">
Background
</h2>
The VariantAnnotation package has facilities for reading in all or subsets of Variant Call Format (VCF) files. These text files contain meta-information lines, a header line and data lines each containing information about a position in the genome. The format also may also contain genotype information on samples for each position. More on the file format can be found in the [VCF specs](http://samtools.github.io/hts-specs/VCFv4.2.pdf).

The 'locateVariants' function in the VariantAnnotation package identifies where a variant is located with respect to the gene model (e.g., exon, intron, splice site, etc.). The 'predictCoding' function reports the amino acid change for non-synonymous coding variants. Consequences of the coding changes can be investigated with the SIFT and PolyPhen database packages. We'll use these functions to learn about variants located on the TRPV gene on chromosome

<h2 id="setup">
Set Up
</h2>
This workflow requires several different Bioconductor packages. Usage of each will be described in detail in the following sections.

In [2]:
## This code chunk was hidden in the original document, but was executed in the background
suppressPackageStartupMessages(library(VariantAnnotation))
suppressMessages(suppressPackageStartupMessages(library(cgdv17)))
suppressPackageStartupMessages(library(org.Hs.eg.db))
suppressPackageStartupMessages(library(TxDb.Hsapiens.UCSC.hg19.knownGene))
suppressPackageStartupMessages(library(BSgenome.Hsapiens.UCSC.hg19))
suppressPackageStartupMessages(library(PolyPhen.Hsapiens.dbSNP131))

``` r
library(VariantAnnotation)
library(cgdv17)
library(org.Hs.eg.db)
library(TxDb.Hsapiens.UCSC.hg19.knownGene)
library(BSgenome.Hsapiens.UCSC.hg19)
library(PolyPhen.Hsapiens.dbSNP131)
```

Use biocLite() to get the packages you don't have installed:

``` r
## try http:// if https:// URLs are not supported
source("https://bioconductor.org/biocLite.R")
biocLite("mypackage")
```

<h2 id="TRPV">
Exploring variants in the TRPV gene family
</h2>
This workflow focuses on variants located in the Transient Receptor Potential Vanilloid (TRPV) gene family on chromosome 17. Sample data are from the Bioconductor cgdv17 experimental data package which contains Complete Genomics Diversity panel data for chromosome 17 on 46 individuals. For more background on how these data were curated see the package vignette.

``` r
browseVignettes("cgdv17")
```

We use a VCF file from the package that is a subset of chromosome 17 for a single individual from the CEU population.

In [3]:
library(VariantAnnotation)
library(cgdv17)
file <- system.file("vcf", "NA06985_17.vcf.gz", package = "cgdv17")

<h3 id="TRPV_header">
Examine header data in a vcf file
</h3>
To get an idea of what data are in the file we look at the header. scanVcfHeader() parses the file header into a VCFHeader object and the info() and geno() accessors extract field-specific data.

In [4]:
hdr <- scanVcfHeader(file)
 
info(hdr) 
 
geno(hdr) 

DataFrame with 3 rows and 3 columns
        Number        Type                 Description
   <character> <character>                 <character>
NS           1     Integer Number of Samples With Data
DP           1     Integer                 Total Depth
DB           0        Flag dbSNP membership, build 131

DataFrame with 12 rows and 3 columns
            Number        Type                         Description
       <character> <character>                         <character>
GT               1      String                            Genotype
GQ               1     Integer                    Genotype Quality
DP               1     Integer                          Read Depth
HDP              2     Integer                Haplotype Read Depth
HQ               2     Integer                   Haplotype Quality
...            ...         ...                                 ...
mRNA             .      String                     Overlaping mRNA
rmsk             .      String                  Overlaping Repeats
segDup           .      String Overlaping segmentation duplication
rCov             1       Float                   relative Coverage
cPd              1      String                called Ploidy(level)

Variants in the VCF have been aligned to NCBI genome build GRCh37:

In [5]:
meta(hdr)$META

DataFrame with 5 rows and 1 column
                      Value
                <character>
fileformat          VCFv4.1
fileDate           20111102
source     masterVar2VCFv40
reference    build37.fa.bz2
phasing             partial

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h3 id="TRPV_symbols">
Convert gene symbols to gene ids
</h3>
Use the org.Hs.eg.db package to convert gene symbols to gene ids.

In [6]:
## get entrez ids from gene symbols
library(org.Hs.eg.db)
genesym <- c("TRPV1", "TRPV2", "TRPV3")
geneid <- select(org.Hs.eg.db, keys=genesym, keytype="SYMBOL",
                 columns="ENTREZID")
geneid

'select()' returned 1:1 mapping between keys and columns


,SYMBOL,ENTREZID
1,TRPV1,7442
2,TRPV2,51393
3,TRPV3,162514


<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h3 id="TRPV_ranges">
Create gene ranges
</h3>
We use the hg19 known gene track from UCSC to identify the TRPV gene ranges. These ranges will eventually be used to extract variants from a regions in the VCF file.

Load the annotation package.

In [7]:
library(TxDb.Hsapiens.UCSC.hg19.knownGene)
txdb <- TxDb.Hsapiens.UCSC.hg19.knownGene
txdb

TxDb object:
# Db type: TxDb
# Supporting package: GenomicFeatures
# Data source: UCSC
# Genome: hg19
# Organism: Homo sapiens
# Taxonomy ID: 9606
# UCSC Table: knownGene
# Resource URL: http://genome.ucsc.edu/
# Type of Gene ID: Entrez Gene ID
# Full dataset: yes
# miRBase build ID: GRCh37
# transcript_nrow: 82960
# exon_nrow: 289969
# cds_nrow: 237533
# Db created by: GenomicFeatures package from Bioconductor
# Creation time: 2015-10-07 18:11:28 +0000 (Wed, 07 Oct 2015)
# GenomicFeatures version at creation time: 1.21.30
# RSQLite version at creation time: 1.0.0
# DBSCHEMAVERSION: 1.1

Our VCF file was aligned to a genome from NCBI and the known gene track was from UCSC. These institutions have different naming conventions for the chromosomes. In order to use these two pieces of data in a matching or overlap operation the chromosome names (also called sesqlevels) need to match. We will modify the txdb to match the VCF file.

In [8]:
txdb <- renameSeqlevels(txdb, gsub("chr", "", seqlevels(txdb)))
txdb <- keepSeqlevels(txdb, "17")

Create a list of transcripts by gene:

In [9]:
txbygene = transcriptsBy(txdb, "gene")

Create the gene ranges for the TRPV genes

In [10]:
gnrng <- unlist(range(txbygene[geneid$ENTREZID]), use.names=FALSE)
names(gnrng) <- geneid$SYMBOL

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h3 id="TRPV_subset">
Extract variant subsets
</h3>
A ScanVcfParam object is used to retrieve data subsets. This object can specify genomic coordinates (ranges) or individual VCF elements. Extractions of ranges (vs fields) requires a tabix index. See ?indexTabix for details.

In [11]:
param <- ScanVcfParam(which = gnrng, info = "DP", geno = c("GT", "cPd"))
param
 
## Extract the TRPV ranges from the VCF file 
vcf <- readVcf(file, "hg19", param)
## Inspect the VCF object with the 'fixed', 'info' and 'geno' accessors
vcf
 
head(fixed(vcf))

geno(vcf)

class: ScanVcfParam 
vcfWhich: 1 elements
vcfFixed: character() [All] 
vcfInfo: DP 
vcfGeno: GT cPd 
vcfSamples:  

class: CollapsedVCF 
dim: 405 1 
rowRanges(vcf):
  GRanges with 5 metadata columns: paramRangeID, REF, ALT, QUAL, FILTER
info(vcf):
  DataFrame with 1 column: DP
info(header(vcf)):
      Number Type    Description
   DP 1      Integer Total Depth
geno(vcf):
  SimpleList of length 2: cPd, GT
geno(header(vcf)):
       Number Type   Description         
   cPd 1      String called Ploidy(level)
   GT  1      String Genotype            

DataFrame with 6 rows and 4 columns
             REF                ALT      QUAL      FILTER
  <DNAStringSet> <DNAStringSetList> <numeric> <character>
1              A                  G       120        PASS
2              A                            0        PASS
3          AAAAA                            0        PASS
4             AA                            0        PASS
5              C                  T        59        PASS
6              T                  C       157        PASS

List of length 2
names(2): cPd GT

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h3 id="TRPV_locate">
Variant location in the gene model
</h3>
The locateVariants function identifies where a variant falls with respect to gene structure, e.g., exon, utr, splice site, etc. We use the gene model from the TxDb.Hsapiens.UCSC.hg19.knownGene package loaded eariler.

``` r
## Use the 'region' argument to define the region
## of interest. See ?locateVariants for details.
cds <- locateVariants(vcf, txdb, CodingVariants())
five <- locateVariants(vcf, txdb, FiveUTRVariants())
splice <- locateVariants(vcf, txdb, SpliceSiteVariants())
intron <- locateVariants(vcf, txdb, IntronVariants())
```

In [12]:
all <- locateVariants(vcf, txdb, AllVariants())

'select()' returned many:1 mapping between keys and columns
'select()' returned many:1 mapping between keys and columns
'select()' returned many:1 mapping between keys and columns
'select()' returned many:1 mapping between keys and columns
'select()' returned many:1 mapping between keys and columns
'select()' returned many:1 mapping between keys and columns


Each row in cds represents a variant-transcript match so multiple rows per variant are possible. If we are interested in gene-centric questions the data can be summarized by gene regardless of transcript.

In [13]:
## Did any variants match more than one gene?
table(sapply(split(mcols(all)$GENEID, mcols(all)$QUERYID), 
      function(x) length(unique(x)) > 1))
 
## Summarize the number of variants by gene:
idx <- sapply(split(mcols(all)$QUERYID, mcols(all)$GENEID), unique)
sapply(idx, length)
 
## Summarize variant location by gene:
sapply(names(idx), 
    function(nm) {
        d <- all[mcols(all)$GENEID %in% nm, c("QUERYID", "LOCATION")]
        table(mcols(d)$LOCATION[duplicated(d) == FALSE])
    })


FALSE  TRUE 
  367    38 

125144 162514  23729  51393   7442  84690 
     1    196      2     63    146     35

,125144,162514,23729,51393,7442,84690
spliceSite,0,2,0,0,1,0
intron,0,153,0,58,117,19
fiveUTR,0,2,0,1,3,5
threeUTR,0,24,2,1,2,0
coding,0,5,0,3,8,0
intergenic,0,0,0,0,0,0
promoter,1,10,0,0,15,11


<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h3 id="TRPV_predict">
Amino acid coding changes in non-synonymous variants
</h3>
Amino acid coding for non-synonymous variants can be computed with the function predictCoding. The BSgenome.Hsapiens.UCSC.hg19 package is used as the source of the reference alleles. Variant alleles are provided by the user.

In [14]:
library(BSgenome.Hsapiens.UCSC.hg19)
seqlevelsStyle(vcf) <- "UCSC"
seqlevelsStyle(txdb) <- "UCSC"
aa <- predictCoding(vcf, txdb, Hsapiens)

Warning message:
In .predictCodingGRangesList(query, cache[["cdsbytx"]], seqSource, : records with missing 'varAllele' were ignoredWarning message:
In .predictCodingGRangesList(query, cache[["cdsbytx"]], seqSource, : 'varAllele' values with 'N', '.', '+' or '-' were not translated

predictCoding returns results for coding variants only. As with locateVariants, the output has one row per variant-transcript match so multiple rows per variant are possible.

In [15]:
## Did any variants match more than one gene?
table(sapply(split(mcols(aa)$GENEID, mcols(aa)$QUERYID), 
        function(x) length(unique(x)) > 1))

## Summarize the number of variants by gene:
idx <- sapply(split(mcols(aa)$QUERYID, mcols(aa)$GENEID, drop=TRUE), unique)
sapply(idx, length)

## Summarize variant consequence by gene:
sapply(names(idx), 
       function(nm) {
           d <- aa[mcols(aa)$GENEID %in% nm, c("QUERYID","CONSEQUENCE")]
           table(mcols(d)$CONSEQUENCE[duplicated(d) == FALSE])
       })


FALSE 
   17 

162514  51393   7442 
     6      3      8

,162514,51393,7442
nonsynonymous,2,0,2
not translated,1,0,5
synonymous,3,3,1


The variants 'not translated' are explained by the warnings thrown when predictCoding was called. Variants that have a missing varAllele or have an 'N' in the varAllele are not translated. If the varAllele substitution had resulted in a frameshift the consequence would be 'frameshift'. See ?predictCoding for details.

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h2 id="vep">
Annotating with the ensemblVEP package
</h2>
The ensemblVEP package provides access to the online Ensembl Variant Effect Predictor (VEP tool). The VEP tool ouputs predictions of functional consequences of known and unknown variants as reported by Sequence Ontology or Ensembl. Regulatory region consequences, HGNC, Ensembl protein identifiers, HGVS, co-located variants are optional outputs. ensemblVEP() accepts the name of a VCF file and returns a VCF on disk or GRanges in the R workspace.

In [16]:
## This code chunk was hidden in the original document, but was executed in the background
suppressPackageStartupMessages(library(ensemblVEP))

ERROR: Error in library(ensemblVEP): there is no package called ‘ensemblVEP’


Load ensemblVEP:

In [17]:
library(ensemblVEP)

ERROR: Error in library(ensemblVEP): there is no package called ‘ensemblVEP’


The 'file' argument to ensemblVEP must be a vcf file on disk. We'll write out the VCF object with the TRPV variants and submit that to ensemblVEP.

Write the VCF object to a vcf file in tempfile():

In [18]:
dest <- tempfile()
writeVcf(vcf, dest)

Call ensemblVEP with the file containing only the TRPV variants and the custom VEPParam object:

``` r
gr <- ensemblVEP(file = dest)
```

``` r
2015-06-09 08:50:22 - Starting...
2015-06-09 08:50:24 - Detected format of input file as vcf
2015-06-09 08:50:24 - Read 277 variants into buffer
2015-06-09 08:50:24 - Reading transcript data from cache and/or database
[================================================================================================================================================================]  [ 100% ]
2015-06-09 08:54:29 - Retrieved 55 transcripts (0 mem, 0 cached, 55 DB, 0 duplicates)
2015-06-09 08:54:29 - Analyzing chromosome 17
2015-06-09 08:54:29 - Analyzing variants
[================================================================================================================================================================]  [ 100% ]
2015-06-09 08:54:29 - Calculating consequences
[================================================================================================================================================================]  [ 100% ]
2015-06-09 08:54:30 - Processed 277 total variants (1 vars/sec, 1 vars/sec total)
2015-06-09 08:54:30 - Wrote stats summary to /tmp/RtmppUYah1/file339f64bf5700_summary.html
2015-06-09 08:54:30 - Finished!
```

``` r
head(gr, 3)
```

``` r
GRanges object with 3 ranges and 22 metadata columns:
           seqnames             ranges strand |   Allele
              <Rle>          <IRanges>  <Rle> | <factor>
  rs402369    chr17 [3469401, 3469401]      * |        G
  rs402369    chr17 [3469401, 3469401]      * |        G
  rs402369    chr17 [3469401, 3469401]      * |        G
                                    Consequence   IMPACT   SYMBOL
                                       <factor> <factor> <factor>
  rs402369 splice_region_variant&intron_variant      LOW  SPATA22
  rs402369 splice_region_variant&intron_variant      LOW  SPATA22
  rs402369                upstream_gene_variant MODIFIER     ASPA
                      Gene Feature_type         Feature        BIOTYPE     EXON
                  <factor>     <factor>        <factor>       <factor> <factor>
  rs402369 ENSG00000141255   Transcript ENST00000397168 protein_coding     <NA>
  rs402369 ENSG00000141255   Transcript ENST00000355380 protein_coding     <NA>
  rs402369 ENSG00000108381   Transcript ENST00000456349 protein_coding     <NA>
             INTRON    HGVSc    HGVSp cDNA_position CDS_position
           <factor> <factor> <factor>      <factor>     <factor>
  rs402369      1/8     <NA>     <NA>          <NA>         <NA>
  rs402369      1/7     <NA>     <NA>          <NA>         <NA>
  rs402369     <NA>     <NA>     <NA>          <NA>         <NA>
           Protein_position Amino_acids   Codons Existing_variation DISTANCE
                   <factor>    <factor> <factor>           <factor> <factor>
  rs402369             <NA>        <NA>     <NA>               <NA>     <NA>
  rs402369             <NA>        <NA>     <NA>               <NA>     <NA>
  rs402369             <NA>        <NA>     <NA>               <NA>     4652
             STRAND SYMBOL_SOURCE    HGNC_ID
           <factor>      <factor>   <factor>
  rs402369       -1          HGNC HGNC:30705
  rs402369       -1          HGNC HGNC:30705
  rs402369        1          HGNC   HGNC:756
  -------
  seqinfo: 25 sequences from  genome; no seqlengths
```

Data from the VEP tool is returned as metadata columns in the GRanges ('gr' object). You can further control what fields are returned by setting the runtime options in the VEPParam.

In [19]:
VEPParam()

ERROR: Error in eval(expr, envir, enclos): could not find function "VEPParam"


Each of the options, e.g., 'basic', 'inut', 'cache' etc. has multiple fields that can be set.

In [20]:
basic(VEPParam())

ERROR: Error in eval(expr, envir, enclos): could not find function "basic"


For more details on runtime options see the ?VEPParam man page and the [Ensembl VEP web site](http://uswest.ensembl.org/info/docs/tools/vep/script/vep_options.html).

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h2 id="content">
Exploring Package Content
</h2>
Packages have extensive help pages, and include vignettes highlighting common use cases. The help pages and vignettes are available from within R. After loading a package, use syntax like

In [21]:
help(package="VariantAnnotation")
?predictCoding

predictCoding {VariantAnnotation},R Documentation
query,"A VCF, Ranges, GRanges or VRanges instance containing the variants to be annotated. If query is a Ranges or VRanges it is coerced to a GRanges. If a VCF is provided the GRanges returned by the rowRanges() accessor will be used. All metadata columns are ignored. When query is not a VCF object a varAllele must be provided. The varAllele must be a DNAStringSet the same length as the query. If there are multiple alternate alleles per variant the query must be expanded to one row per each alternate allele. See examples. NOTE: Variants are expected to conform to the VCF specs as described on the 1000 Genomes page (see references). Indels must include the reference allele; zero-width ranges are not valid and return no matches."
subject,A TxDb object that serves as the annotation. GFF files can be converted to TxDb objects with makeTxDbFromGFF() in the GenomicFeatures package.
seqSource,A BSgenome instance or an FaFile to be used for sequence extraction.
varAllele,A DNAStringSet containing the variant (alternate) alleles. The length of varAllele must equal the length of query. Missing values are represented by a zero width empty element. Ranges with missing varAllele values are ignored; those with an ‘N’ character are not translated. When the query is a VCF object the varAllele argument will be missing. This value is taken internally from the VCF with alt(<VCF>).
...,Additional arguments passed to methods. Arguments genetic.code and if.fuzzy.codon are supported for the translate function.
ignore.strand,"A logical indicating if strand should be ignored when performing overlaps. When ignore.strand == TRUE the query strand is set to '*' and can overlap with any strand of the subject. The return GRanges reflects the strand of the subject hit, however, the positions and alleles reported are computed as if both were from the '+' strand. ignore.strand == FALSE requires the query and subject to have compatible strand. Again the return GRanges reports the strand of the subject hit but in this case positions and alleles are computed according to the strand of the subject."


to obtain an overview of help on the `VariantAnnotation` package, and the `predictCoding` function. View the package vignette with

``` r
browseVignettes(package="VariantAnnotation")
```

To view vignettes providing a more comprehensive introduction to package functionality use

``` r
help.start()
```

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h2 id="session">
sessionInfo()
</h2>

In [22]:
sessionInfo()

R version 3.2.2 (2015-08-14)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 14.04.3 LTS

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=de_DE.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=de_DE.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=de_DE.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    parallel  stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] PolyPhen.Hsapiens.dbSNP131_1.0.2       
 [2] BSgenome.Hsapiens.UCSC.hg19_1.4.0      
 [3] BSgenome_1.38.0                        
 [4] rtracklayer_1.30.1                     
 [5] TxDb.Hsapiens.UCSC.hg19.knownGene_3.2.2
 [6] GenomicFeatures_1.22.6                 
 [7] org.Hs.eg.db_3.2.3                     
 [8] RSQLite_1.0.0                          
 [9] DBI_0.3.1            

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>